In [1]:
from pyvis.network import Network
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from collections import Counter
import os

# Cargar el DataFrame desde un archivo CSV
# Asegúrate de que el archivo CSV esté en el mismo directorio que tu notebook
d5 = pd.read_csv('datos.csv')

# Crear una función para generar la visualización del grafo basada en un criterio de filtrado y ajustes dinámicos
def generar_grafo(facultad_filtro, carrera_filtro, autor_filtro, node_size, text_size, node_width):
    net = Network(notebook=True, height='750px', width='100%', bgcolor='#ffffff', font_color='black', cdn_resources='in_line')

    # Diccionario de colores para las facultades
    facultad_colors = {
        'Facultad de Arquitectura y Urbanismo': 'red',
        'No PUCV': 'gray',
        'Facultad de Ciencias': 'blue',
        'Facultad de Ingeniería': 'green',
        'Facultad de Ciencias Económicas y Administrativas': 'orange',
        'Facultad de Filosofía y Educación': 'purple',
        'Facultad de Ciencias Agronómicas y de los Alimentos': 'yellow',
        'Facultad de Derecho': 'brown',
        'Vicerrectoría de Estudios Avanzados PUCV': 'magenta',
        'Ciencias Económicas y Administrativas': 'cyan'
    }

    # Filtrar los datos según los criterios seleccionados
    datos_filtrados = d5
    if facultad_filtro != 'Todos':
        datos_filtrados = datos_filtrados[datos_filtrados['Facultad'] == facultad_filtro]
    if carrera_filtro != 'Todos':
        datos_filtrados = datos_filtrados[datos_filtrados['Carrera'] == carrera_filtro]
    if autor_filtro != 'Todos':
        datos_filtrados = datos_filtrados[datos_filtrados['Autor(es)'].str.contains(autor_filtro)]

    # Asegurar que los valores en la columna 'Autor(es)' sean cadenas de texto
    datos_filtrados['Autor(es)'] = datos_filtrados['Autor(es)'].fillna('').astype(str)

    # Función para verificar si un nodo ya existe
    def node_exists(net, node_id):
        node_id = str(node_id)  # Asegurarse de que node_id es un string
        for node in net.nodes:
            if node['id'] == node_id:
                return True
        return False

    # Agregar nodos para autores y títulos
    for _, row in datos_filtrados.iterrows():
        titulo = row['Título']
        autores = row['Autor(es)'].split(',')  # Asumimos que los autores están separados por comas
        universidad = row['Universidad']
        facultad = row['Facultad']
        unidad = row['Unidad']
        carrera = row['Carrera']
        nombre_proyecto = row['Nombre Proyecto']

        for autor in autores:
            autor = autor.strip()
            if not node_exists(net, autor):
                facultad_color = facultad_colors.get(facultad, 'gray')  # Usar gris si la facultad no está en el diccionario
                net.add_node(str(autor), label=autor, title=f"Universidad: {universidad}\nUnidad: {unidad}", color=facultad_color, size=node_size)

        if not node_exists(net, titulo):
            # Ajustar el tamaño del nodo y el ancho del nodo basado en el contenido
            net.add_node(str(titulo), label=titulo, title=f"Título: {titulo}\nCarrera: {carrera}\nNombre Proyecto: {nombre_proyecto}", shape='box', size=node_size, font={'size': text_size}, widthConstraint=node_width, color={'background': 'orange', 'border': 'yellow'})

    # Agregar aristas para publicaciones
    for _, row in datos_filtrados.iterrows():
        titulo = row['Título']
        autores = row['Autor(es)'].split(',')  # Asumimos que los autores están separados por comas
        categoria_actividad = row['Categoría actividad']

        dashes = True if categoria_actividad == 'Yes' else False
        for autor in autores:
            net.add_edge(str(autor.strip()), str(titulo), title=f"Categoría actividad: {categoria_actividad}\nTítulo: {titulo}", dashes=dashes)

    # Guardar el gráfico en un archivo temporal y cargar su contenido
    temp_file_path = 'grafo_publicaciones.html'
    net.show(temp_file_path)
    with open(temp_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Mostrar el contenido del archivo HTML en el notebook sin imprimir la ruta del archivo
    display(HTML(html_content))

# Crear widgets de selección para las facultades, carreras y autores
facultades = ['Todos'] + list(d5['Facultad'].unique())
carreras = ['Todos'] + list(d5['Carrera'].unique())

# Contar la recurrencia de cada autor
autor_counts = Counter([autor.strip() for autores in d5['Autor(es)'].fillna('').astype(str).str.split(',') for autor in autores])
autores = ['Todos'] + [autor for autor, _ in autor_counts.most_common()]

dropdown_facultad = widgets.Dropdown(options=facultades, description='Facultad:', disabled=False)
dropdown_carrera = widgets.Dropdown(options=carreras, description='Carrera:', disabled=False)
dropdown_autor = widgets.Combobox(options=autores, description='Autor:', ensure_option=True, disabled=False)

# Crear widgets para ajustar las opciones de visualización con valores por defecto
slider_node_size = widgets.IntSlider(value=19, min=10, max=100, step=1, description='Tamaño Nodo:', continuous_update=False)
slider_text_size = widgets.IntSlider(value=20, min=10, max=50, step=1, description='Tamaño Texto:', continuous_update=False)
slider_node_width = widgets.IntSlider(value=140, min=100, max=500, step=10, description='Ancho Nodo:', continuous_update=False)

# Disponer los widgets en una fila horizontal
filters_box = widgets.HBox([dropdown_facultad, dropdown_carrera, dropdown_autor])
sliders_box = widgets.HBox([slider_node_size, slider_text_size, slider_node_width])

# Conectar los widgets al gráfico manualmente usando interactive
ui = widgets.VBox([filters_box, sliders_box])
out = widgets.interactive_output(generar_grafo, {
    'facultad_filtro': dropdown_facultad,
    'carrera_filtro': dropdown_carrera,
    'autor_filtro': dropdown_autor,
    'node_size': slider_node_size,
    'text_size': slider_text_size,
    'node_width': slider_node_width
})

# Mostrar los widgets y el gráfico
display(ui, out)

# Añadir el glosario de colores de las facultades
glosario = """
<h3>Glosario de colores por facultad:</h3>
<ul>
    <li><span style="color:red;">■</span> Facultad de Arquitectura y Urbanismo</li>
    <li><span style="color:gray;">■</span> No PUCV</li>
    <li><span style="color:blue;">■</span> Facultad de Ciencias</li>
    <li><span style="color:green;">■</span> Facultad de Ingeniería</li>
    <li><span style="color:orange;">■</span> Facultad de Ciencias Económicas y Administrativas</li>
    <li><span style="color:purple;">■</span> Facultad de Filosofía y Educación</li>
    <li><span style="color:yellow;">■</span> Facultad de Ciencias Agronómicas y de los Alimentos</li>
    <li><span style="color:brown;">■</span> Facultad de Derecho</li>
    <li><span style="color:magenta;">■</span> Vicerrectoría de Estudios Avanzados PUCV</li>
    <li><span style="color:cyan;">■</span> Ciencias Económicas y Administrativas</li>
</ul>
"""

# Mostrar el glosario
display(HTML(glosario))


ModuleNotFoundError: No module named 'pyvis'